In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!pip install apache_beam mwparserfromhell

In [ ]:
!pip install datasets

In [106]:
## public libraries
from datasets import load_dataset

## local libraries
from gpt_utils import (
    get_encoder_decoder_size,
    text_to_tv_tensors,
    get_batch,
    training_run
)
from attention_decoder import AttentionModule


In [2]:
wiki_raw = load_dataset("wikipedia", "20220301.simple")

Found cached dataset wikipedia (/Users/paul/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
wiki_raw

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 205328
    })
})

In [6]:
wiki_text_blob = '\n\n'.join(wiki_raw['train']['text'])
print(len(wiki_text_blob))
print()
print(wiki_text_blob[:1000])

215900536

April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.

April always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.

April's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.

The Month 

April comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.

April begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.

In common years, April starts on the same day of the week as October of the previous year, and in le

In [5]:
##When sorted, low-numbered codes seem to be more "normal"
normal_chars = ''.join(sorted(list(set(wiki_text_blob)))[0:97])
normal_chars

'\t\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~'

In [7]:
# 30 sec ish
wiki_text_blob_clean = ''.join([x for x in wiki_text_blob if x in normal_chars])

In [8]:
# Keep 99.7% of chars. great.
len(wiki_text_blob_clean)/len(wiki_text_blob)

0.997293827931951

In [25]:
encode, decode, vocab_size = get_encoder_decoder_size(wiki_text_blob_clean)
print(vocab_size)

97


In [26]:
train, validate = text_to_tv_tensors(wiki_text_blob_clean, encode)

torch.Size([215316272]) torch.int64
tensor([35, 82, 84, 75, 78,  2, 75, 85,  2, 86, 74, 71,  2, 72, 81, 87, 84, 86,
        74,  2, 79, 81, 80, 86, 74,  2, 81, 72,  2, 86, 74, 71,  2, 91, 71, 67,
        84,  2, 75, 80,  2, 86, 74, 71,  2, 44, 87, 78, 75, 67, 80,  2, 67, 80,
        70,  2, 41, 84, 71, 73, 81, 84, 75, 67, 80,  2, 69, 67, 78, 71, 80, 70,
        67, 84, 85, 14,  2, 67, 80, 70,  2, 69, 81, 79, 71, 85,  2, 68, 71, 86,
        89, 71, 71, 80,  2, 47, 67, 84, 69, 74])


In [47]:
m_attn = AttentionModule(vocab_size)

In [109]:
training_run(m_attn, train, validate, num_epochs=10000)

Training for 10000 Epochs...
Epoch: 0, Train Loss: 2.2303, Val Loss: 2.2419
Epoch: 500, Train Loss: 2.2137, Val Loss: 2.2528
Epoch: 1000, Train Loss: 2.2072, Val Loss: 2.2454
Epoch: 1500, Train Loss: 2.1940, Val Loss: 2.2141
Epoch: 2000, Train Loss: 2.1929, Val Loss: 2.2323
Epoch: 2500, Train Loss: 2.1914, Val Loss: 2.2181
Epoch: 3000, Train Loss: 2.1742, Val Loss: 2.1857
Epoch: 3500, Train Loss: 2.1596, Val Loss: 2.1830
Epoch: 4000, Train Loss: 2.1692, Val Loss: 2.1752
Epoch: 4500, Train Loss: 2.1565, Val Loss: 2.1794
Epoch: 5000, Train Loss: 2.1634, Val Loss: 2.1729
Epoch: 5500, Train Loss: 2.1304, Val Loss: 2.1764
Epoch: 6000, Train Loss: 2.1404, Val Loss: 2.1598
Epoch: 6500, Train Loss: 2.1320, Val Loss: 2.1436
Epoch: 7000, Train Loss: 2.1351, Val Loss: 2.1484
Epoch: 7500, Train Loss: 2.1086, Val Loss: 2.1503
Epoch: 8000, Train Loss: 2.1100, Val Loss: 2.1609
Epoch: 8500, Train Loss: 2.1371, Val Loss: 2.1374
Epoch: 9000, Train Loss: 2.0993, Val Loss: 2.1152
Epoch: 9500, Train Loss: 

In [51]:
seed_raw = "To be or not to "
seed = torch.tensor(encode(seed_raw)).view(1,-1)

print(decode(m_attn.generate(seed, 1000)[0,]))

To be or not to thip cask RIst 193 205
 201953 pror ynspereanis. c Toustars anfercs Destttinings Mibeetgauet ohe he nasies  19" is al ins sp and birias Baus nthisidanaret Hapstt and modd they and opece eont Glatrsh, Epanacals Jucky

19
is beostizitirebus) He brafitarar acmes of futrsg.
thed Slony nose ahaus thing warves  greatson" in atsthoos: ale  w'irnctar ding an F he wnoly
 Fremip 19 dider, Fury ath wishel whithiriflis wingl wifth the myypersana.
 Ref Mavatos, and uso Bucknealie San gies the sos cat su., Noat couf Das"|216 T||||0|3)

|0,) ane Rif Morcen, cantlerian sononters, Domun wos gillsst. It Janmong a Fe bules the  uffam Nese nand, ame pantie (69).16, 1984 omec.
 Ire Kulgzss wragite
 Therolt iflerd lidfies thehe wed by
 Haurgolst (ion 20).


T638970 sthe rock the bHos, Vien canade 19  neccrdiners the the arrad wiccert bewhama
Brrtht sham the Ssemenilus eland. Is in ofky Al Thaca i is by Pesrisingituge sonetu, ar cong ald Kude.


Hugersi

Jalainge Wimmad


29< brck Mweresstseb

In [81]:
from statistics import mean

In [107]:
m_attn.eval()
train_batches = [get_batch(train) for _ in range(100)]
#model returns logits, loss
train_losses = [m_attn(x[0], x[1])[1].item() for x in train_batches]

m_attn.train() #goes back to train mode

print(f"Train Loss: {mean(train_losses):.4f}")

Train Loss: 2.2655


In [87]:
x = train_batches[0]

In [97]:
m_attn(x[0], x[1])[1].item()

2.388345718383789

In [77]:
tx

tensor([[71, 84, 81, 80, 75, 77, 67,  2, 47, 67, 84, 69, 74, 71, 80, 77],
        [80, 71, 14,  2, 81, 90, 91, 73, 71, 80, 14,  2, 80, 75, 86, 84],
        [84, 91,  2, 84, 71, 85, 82, 71, 69, 86, 75, 88, 71, 78, 91, 16],
        [71, 16,  2, 49, 80,  2, 19, 24,  2, 44, 67, 80, 87, 67, 84, 91],
        [27, 25, 26,  2, 74, 71,  2, 89, 67, 85,  2, 67, 85, 85, 75, 73],
        [51, 87, 71, 71, 80,  2, 81, 72,  2, 86, 74, 71,  2, 48, 71, 86],
        [ 2, 81, 82, 71, 80, 85,  2, 81, 80,  2, 86, 74, 71,  2, 85, 75],
        [80, 67,  2, 36, 67, 68, 87,  2, 50, 67, 87, 78,  2, 10, 48, 75],
        [16,  2, 42, 75, 85,  2, 79, 67, 86, 71, 84, 80, 67, 78,  2, 73],
        [10, 19, 27, 26, 19, 11,  2, 85, 87, 84, 88, 71, 91, 71, 70,  2],
        [74, 75, 85,  2, 37, 75, 86, 91,  4,  2, 89, 67, 85,  2, 84, 71],
        [71, 85, 85,  2, 47, 67, 84, 75, 67,  2, 53, 69, 74, 71, 78, 78],
        [70, 75, 80, 73,  2, 82, 81, 85, 86, 85, 71, 67, 85, 81, 80, 11],
        [80,  2, 71, 70, 87, 69, 67, 8